chat completion model


In [2]:
print("Langchain 2")

Langchain 2


In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from langchain_groq import ChatGroq
GROQ_API_KEY=os.environ["GROQ_API_KEY"]

In [6]:

# Initialize the Groq LLM with the model name and parameters
llamaChatModel = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama3-70b-8192",
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
)

Prompts and Prompt Templates


In [7]:
# chat completion model

from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an {profession} expert on {topic}."),
        ("human", "Hello, Mr. {profession}, can you please answer a question?"),
        ("ai", "Sure!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(
    profession="Historian",
    topic="The Kennedy family",
    user_input="How many grandchildren had Joseph P. Kennedy?"
)

response = llamaChatModel.invoke(messages)
print(response.content)

Joseph P. Kennedy, the patriarch of the Kennedy family, had a total of 29 grandchildren. Yes, you heard that right - 29! His children, including John F. Kennedy, Robert F. Kennedy, Ted Kennedy, and the others, had a large brood of kids, and many of them have continued to make their mark on American politics, business, and society.


Old way


In [9]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are an Historian expert on the Kennedy family."
            )
        ),
        HumanMessagePromptTemplate.from_template("{user_input}"),
    ]
)

messages = chat_template.format_messages(
    user_input="Name the children and grandchildren of Joseph P. Kennedy?"
)

response = llamaChatModel.invoke(messages)

In [10]:
print(response.content)

As an historian expert on the Kennedy family, I'd be delighted to provide you with the list of children and grandchildren of Joseph P. Kennedy Sr.:

**Children of Joseph P. Kennedy Sr. and Rose Fitzgerald Kennedy:**

1. Joseph Patrick Kennedy Jr. (1915-1944)
2. John Fitzgerald Kennedy (1917-1963) - 35th President of the United States
3. Rosemary Kennedy (1918-2005)
4. Kathleen Agnes Kennedy (1920-1948)
5. Eunice Mary Kennedy (1921-2009)
6. Patricia Helen Kennedy (1924-2006)
7. Robert Francis Kennedy (1925-1968) - U.S. Attorney General and U.S. Senator
8. Jean Ann Kennedy (b. 1928)
9. Edward Moore Kennedy (1932-2009) - U.S. Senator

**Grandchildren of Joseph P. Kennedy Sr.:**

**Children of Joseph P. Kennedy Jr. (no children, as he died in 1944)**

**Children of John F. Kennedy:**

1. Arabella Kennedy (1956-1956) - stillborn
2. Caroline Bouvier Kennedy (b. 1957)
3. John Fitzgerald Kennedy Jr. (1960-1999)

**Children of Rosemary Kennedy (no children)**

**Children of Kathleen Kennedy:**


Few Shot Prompting


In [12]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
response= llamaChatModel.invoke(final_prompt.format_messages(input="hi?"))
print(response.content)

¿hola?


Chain


In [14]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

chain = final_prompt | llamaChatModel
response = chain.invoke({"input": "hi how are you?"})
print(response.content)

¡hola, cómo estás!


Output Parsers


In [17]:
from langchain_core.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | llamaChatModel | json_parser

d:\anaconda\envs\llmapp\Lib\site-packages\pydantic\_internal\_generate_schema.py:918: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `ResponseSchema` to V2.
  warn(


In [18]:
json_parser.get_format_instructions()

'Return a JSON object.'

In [19]:
res  = json_chain.invoke({"question": "What is the biggest country?"})
print(res)

{'answer': 'Russia'}


In [23]:
llmmodel=ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama3-70b-8192",
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
)


from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "create a jokes!", "output": "Why don't programmers like nature?Because it has too many bugs. "},
    {"input": "create a joke for teachers", "output": "Why did the teacher wear sunglasses in class?Because her students were so bright!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert joke writer."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

son_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | llmmodel | json_parser



d:\anaconda\envs\llmapp\Lib\site-packages\langchain_groq\chat_models.py:370: UserWarning: WARNING! max_completion_tokens is not default parameter.
                    max_completion_tokens was transferred to model_kwargs.
                    Please confirm that max_completion_tokens is what you intended.
  warnings.warn(
d:\anaconda\envs\llmapp\Lib\site-packages\langchain_groq\chat_models.py:370: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


In [26]:
response = json_chain.invoke({"question": "create a joke for teachers"})
print(response)

{'answer': "Why did the teacher bring a ladder to school? Because she wanted to 'elevate' the learning experience!"}
